In [1]:
# Add libraries
library(tidyverse)
library(repr)
library(digest)
library(infer)
library(gridExtra)
library(cowplot)
library(dplyr)
library(broom)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine




In [2]:
# Read data from .csv file
survey <- read.csv("https://raw.githubusercontent.com/Herman-Liao/stat-201-group-project/main/survey%20lung%20cancer.csv")
head(survey)

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC.DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL.CONSUMING,COUGHING,SHORTNESS.OF.BREATH,SWALLOWING.DIFFICULTY,CHEST.PAIN,LUNG_CANCER
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>
1,M,69,1,2,2,1,1,2,1,2,2,2,2,2,2,YES
2,M,74,2,1,1,1,2,2,2,1,1,1,2,2,2,YES
3,F,59,1,1,1,2,1,2,1,2,1,2,2,1,2,NO
4,M,63,2,2,2,1,1,1,1,1,2,1,1,2,2,NO
5,F,63,1,2,1,1,1,1,1,2,1,2,2,1,1,NO
6,F,75,1,2,1,1,2,2,2,2,1,2,2,1,1,YES


In [3]:
# Change most of the variables to boolean variables instead of integer or string variables
survey_2 <- survey %>%
    summarize(gender = GENDER,
              age = AGE,
              smoking = SMOKING - 1 == 1,
              yellow_fingers = YELLOW_FINGERS - 1 == 1,
              anxiety = ANXIETY - 1 == 1,
              chronic_disease = CHRONIC.DISEASE - 1 == 1,
              fatigue = FATIGUE - 1 == 1,
              allergy = ALLERGY - 1 == 1,
              wheezing = WHEEZING - 1 == 1,
              alcohol_consuming = ALCOHOL.CONSUMING - 1 == 1,
              coughing = COUGHING - 1 == 1,
              shortness_of_breath = SHORTNESS.OF.BREATH - 1 == 1,
              swallowing_difficulty = SWALLOWING.DIFFICULTY - 1 == 1,
              chest_pain = CHEST.PAIN - 1 == 1,
              lung_cancer = LUNG_CANCER == "YES")

head(survey_2)

,gender,age,smoking,yellow_fingers,anxiety,chronic_disease,fatigue,allergy,wheezing,alcohol_consuming,coughing,shortness_of_breath,swallowing_difficulty,chest_pain,lung_cancer
,<chr>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
1,M,69,FALSE,TRUE,TRUE,FALSE,TRUE,FALSE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
2,M,74,TRUE,FALSE,FALSE,TRUE,TRUE,TRUE,FALSE,FALSE,FALSE,TRUE,TRUE,TRUE,TRUE
3,F,59,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE,TRUE,TRUE,FALSE,TRUE,FALSE
4,M,63,TRUE,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,TRUE,FALSE
5,F,63,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,TRUE,TRUE,FALSE,FALSE,FALSE
6,F,75,FALSE,TRUE,FALSE,TRUE,TRUE,TRUE,TRUE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE


In [5]:
# Clean and wrangle data; we are only interested in people who have lung cancer and whether or not they smoked and/or consumed alcohol
# We mutate the data this way to properly separate all combinations of smoking and drinking
# We want the difference in proportions for only smoking minus only drinking
survey_clean_wrangled <- survey_2 %>%
    filter(lung_cancer == TRUE) %>%
    select(gender, smoking, alcohol_consuming) %>%
    mutate(only_smoking = smoking & !alcohol_consuming,
           only_drinking = !smoking & alcohol_consuming) %>%
    select(-alcohol_consuming, -smoking) %>%
    filter(!(only_smoking == FALSE & only_drinking == FALSE))

# To convert only_smoking and only_drinking to one variable, make one character variable that says "only smoking" when only_smoking is true, and "only drinking" otherwise
# This works because we only use observations where each patient either only drinks or only smokes, and not neither nor both
survey_clean_wrangled <- survey_clean_wrangled %>%
    mutate(only_smoke_only_drink = ifelse(only_smoking, "only smoking", "only drinking")) %>%
    select(gender, only_smoke_only_drink)

head(survey_clean_wrangled)

,gender,only_smoke_only_drink
,<chr>,<chr>
1,M,only drinking
2,M,only smoking
3,F,only smoking
4,F,only smoking
5,M,only drinking
6,F,only smoking


In [ ]:
# Separate the genders for blocking
survey_clean_wrangled <- survey_clean_wrangled %>%
    group_by(gender)


head(survey_clean_wrangled)

In [ ]:
# Calculate the number of and proportion of only smokers and only drinkers for each replicate
# After calculating the proportions, calculate the difference in proportions
bootstrap_survey_female_summary <- bootstrap_survey_female %>%
    group_by(replicate) %>%
    summarize(num_only_smoking = sum(only_smoking),
              num_only_drinking = sum(only_drinking),
              prop_only_smoking = mean(only_smoking),
              prop_only_drinking = mean(only_drinking),
              diff_in_props = prop_only_smoking - prop_only_drinking)

bootstrap_survey_male_summary <- bootstrap_survey_male %>%
    group_by(replicate) %>%
    summarize(num_only_smoking = sum(only_smoking),
              num_only_drinking = sum(only_drinking),
              prop_only_smoking = mean(only_smoking),
              prop_only_drinking = mean(only_drinking),
              diff_in_props = prop_only_smoking - prop_only_drinking)

head(bootstrap_survey_female_summary)
head(bootstrap_survey_male_summary)

In [ ]:
# Increase the size of future plots
options(repr.plot.width = 13, repr.plot.height = 5)

# Find the sample prop. of smoking only - the prop. of drinking only for female lung cancer patients
sample_diff_in_props_female <- survey_clean_wrangled %>%
    filter(gender == "F") %>%
    select(-gender) %>%
    summarize(prop_only_smoking = mean(only_smoking),
              prop_only_drinking = mean(only_drinking),
              diff_in_props = prop_only_smoking - prop_only_drinking) %>%
    select(diff_in_props) %>%
    pull()

# Plot the bootstrap distribution of smoking only - drinking only for female lung cancer patients;
# the red line shows the sample prop. of smoking only - the prop. of drinking only
bootstrap_distribution_female <- bootstrap_survey_female_summary %>%
    ggplot(aes(x = diff_in_props)) +
        geom_histogram(binwidth = 0.02) +
        geom_vline(xintercept = sample_diff_in_props_female, color = "red") +
        labs(x = "Difference in Proportions (smoking - drinking)") +
        ggtitle("Bootstrap Distribution of Diff. in Props of Female Lung Cancer Patients")

# Find the sample prop. of smoking only - the prop. of drinking only for male lung cancer patients
sample_diff_in_props_male <- survey_clean_wrangled %>%
    filter(gender == "M") %>%
    select(-gender) %>%
    summarize(prop_only_smoking = mean(only_smoking),
              prop_only_drinking = mean(only_drinking),
              diff_in_props = prop_only_smoking - prop_only_drinking) %>%
    select(diff_in_props) %>%
    pull()

# Plot the bootstrap distribution of the prop. of smoking only - the prop. of drinking only for male lung cancer patients;
# the red line shows the sample prop. of smoking only - the prop. of drinking only
bootstrap_distribution_male <- bootstrap_survey_male_summary %>%
    ggplot(aes(x = diff_in_props)) +
        geom_histogram(binwidth = 0.02) +
        geom_vline(xintercept = sample_diff_in_props_male, color = "red") +
        labs(x = "Difference in Proportions (smoking - drinking)") +
        ggtitle("Bootstrap Distribution of Diff. in Props of Male Lung Cancer Patients")

# Display plots above
grid.arrange(bootstrap_distribution_female,
             bootstrap_distribution_male,
             ncol = 2)

In [ ]:
# Set type I error to 5%, can be changed
alpha = 0.05

In [ ]:
# Calculate p-hat and z-score using worksheet 8 section 3.4, then calculate p-value and check if p-value < alpha
# Is captured supposed to be if 0 is included or if sample_diff_in_props_female and sample_diff_in_props_male is included?
bootstrap_survey_female_calcs <- bootstrap_survey_female_summary %>%
    mutate(standard_error = sqrt(prop_only_smoking * (1 - prop_only_smoking) / num_only_smoking + prop_only_drinking * (1 - prop_only_drinking) / num_only_drinking),
           lower_ci = diff_in_props - qnorm(alpha / 2, lower.tail = FALSE) * standard_error,
           upper_ci = diff_in_props + qnorm(alpha / 2, lower.tail = FALSE) * standard_error,
           captured = lower_ci <= 0 & 0 <= upper_ci,
           p_hat = (num_only_smoking * prop_only_smoking + num_only_drinking * prop_only_drinking) / (num_only_smoking + num_only_drinking),
           z_score = diff_in_props / sqrt(p_hat * (1 - p_hat) * (1 / num_only_smoking + 1 / num_only_drinking)),
           p_value = 2 * pnorm(z_score, lower.tail = FALSE),
           reject_null = p_value < alpha)

bootstrap_survey_male_calcs <- bootstrap_survey_male_summary %>%
    mutate(standard_error = sqrt(prop_only_smoking * (1 - prop_only_smoking) / num_only_smoking + prop_only_drinking * (1 - prop_only_drinking) / num_only_drinking),
           lower_ci = diff_in_props - qnorm(alpha / 2, lower.tail = FALSE) * standard_error,
           upper_ci = diff_in_props + qnorm(alpha / 2, lower.tail = FALSE) * standard_error,
           captured = lower_ci <= 0 & 0 <= upper_ci,
           p_hat = (num_only_smoking * prop_only_smoking + num_only_drinking * prop_only_drinking) / (num_only_smoking + num_only_drinking),
           z_score = diff_in_props / sqrt(p_hat * (1 - p_hat) * (1 / num_only_smoking + 1 / num_only_drinking)),
           p_value = 2 * pnorm(z_score, lower.tail = FALSE),
           reject_null = p_value < alpha)

head(bootstrap_survey_female_calcs)
head(bootstrap_survey_male_calcs)

In [ ]:
# Display proportion of times the null hypothesis was rejected
proportion_reject_null_female <- mean(bootstrap_survey_female_calcs$reject_null)
proportion_reject_null_male <- mean(bootstrap_survey_male_calcs$reject_null)

proportion_reject_null_female
proportion_reject_null_male

In [ ]:
options(repr.plot.width = 13, repr.plot.height = 7.5)

# Visualize all of the 95% confidence intervals for the female bootstrapping distributions
many_ci_plot_female <- bootstrap_survey_female_calcs %>%
    ggplot() +
    scale_colour_manual(breaks = c("TRUE", "FALSE"), # Change colour scale for better visibility.
                        values = c("light grey", "black")) +
    geom_segment(aes(x = lower_ci,
                     xend = upper_ci,
                     y = replicate,
                     yend = replicate,
                     colour = "blue")) +
    geom_vline(xintercept = 0, colour = "red", size = 1) +
    labs(title = "1000 95% Confidence Intervals, Female",
         y = "Replicate",
         x = "Difference in Proportions",
         colour = "Captured?") +
    theme_bw() # Sets a theme for better visibility.

# Visualize all of the 95% confidence intervals for the male bootstrapping distributions
many_ci_plot_male <- bootstrap_survey_male_calcs %>%
    ggplot() +
    scale_colour_manual(breaks = c("TRUE", "FALSE"), # Change colour scale for better visibility.
                        values = c("light grey", "black")) +
    geom_segment(aes(x = lower_ci,
                     xend = upper_ci,
                     y = replicate,
                     yend = replicate,
                     colour = "blue")) +
    geom_vline(xintercept = 0, colour = "red", size = 1) +
    labs(title = "1000 95% Confidence Intervals, Male",
         y = "Replicate",
         x = "Difference in Proportions",
         colour = "Captured?") +
    theme_bw() # Sets a theme for better visibility.

# Display plots above
grid.arrange(many_ci_plot_female,
             many_ci_plot_male,
             ncol = 2)

In [ ]:
# Display proportion of times 0 is in the confidence interval
prorportion_captured_female <- mean(bootstrap_survey_female_calcs$captured)
prorportion_captured_male <- mean(bootstrap_survey_male_calcs$captured)

prorportion_captured_female
prorportion_captured_male

2 sample z-test
- $ p_1 $= proportion of only smoker
- $ p_2 $= proportion of only alcohol consumer
- $H_0: p_1 - p_2 = 0$ vs $H_1: p_1 - p_2 \neq 0$

To test $H_0$, we use the following test statistic:

$$
Z = \frac{\hat{p}_1 - \hat{p}_2}{\sqrt{\hat{p}(1-\hat{p})\left(\frac{1}{n_1} + \frac{1}{n_2}\right)}}
$$
where $\hat{p}_1$ and $\hat{p}_2$ are the sample proportions in samples 1 and 2, respectively; $n$ is the sample size; and $\hat{p}$ is the pooled proportion, given by: 

$$\hat{p} = \frac{n_1\hat{p}_1+n_2\hat{p}_2}{n_1+n_2}$$ from(worksheet 8)

In [ ]:
# calculate p1 and p2 (female)
survey_summary_clean_wrangled <- survey_clean_wrangled %>%
    group_by(gender) %>%
    summarize(prop_smoking = mean(only_smoking == TRUE),
              prop_alcohol = mean(only_drinking == TRUE))

# Summarize number of smokers, drinkers, people who did both, and people who did neither
survey_num_observations_clean_wrangled <- survey_clean_wrangled %>%
    group_by(gender) %>%
    summarize(only_smoking_num = sum(only_smoking == TRUE),
              only_alcohol_consuming_num = sum(only_drinking== TRUE))

survey_summary_clean_wrangled 
survey_num_observations_clean_wrangled

In [ ]:
#calculate test statistic for female
p1_female <- survey_summary_clean_wrangled$prop_smoking[1]
p2_female <- survey_summary_clean_wrangled$prop_alcohol[1]
n1_female <- survey_num_observations_clean_wrangled$only_smoking_num[1]
n2_female <- survey_num_observations_clean_wrangled$only_alcohol_consuming_num[1]

p_hat_female <-  (n1_female * p1_female + n2_female * p2_female) / (n1_female + n2_female) # calculate pooled proportion
t.s_female <- z_score_male <- (p1_female - p2_female) / sqrt(p_hat_female * (1 - p_hat_female) * (1 / n1_female + 1 / n2_female))#calculate the test statistic

p_hat_female
t.s_female

In [ ]:
#calculate the probabilty when significance level is 0.5
p_value_female <- pnorm(t.s_female, lower.tail = FALSE)
p_value_female

since 0.0513 > 0.05, we fail to reject the null hypothesis, so drinking alcohol and smoking have the samp

In [ ]:
#calculate test statistic for male
p1_male <- survey_summary_clean_wrangled$prop_smoking[2]
p2_male <- survey_summary_clean_wrangled$prop_alcohol[2]
n1_male <- survey_num_observations_clean_wrangled$only_smoking_num[2]
n2_male <- survey_num_observations_clean_wrangled$only_alcohol_consuming_num[2]

p_hat_male <-  (n1_male * p1_male + n2_male * p2_male) / (n1_male + n2_male) # calculate pooled proportion
t.s_male <- z_score_male <- (p1_male - p2_male) / sqrt(p_hat_male * (1 - p_hat_male) * (1 / n1_male + 1 / n2_male))#calculate the test statistic

p_hat_male
t.s_male

In [ ]:
#calculate the probabilty when significance level is 0.5
p_value_male <- pnorm(t.s_male, lower.tail = FALSE)
p_value_male

since 0.96 > 0.05, we fail to reject the null hypothesis, there is no difference in proportion between